# Mammography data Analysis

## load data

### Conert data to csv

In [40]:
import pandas as pd

In [ ]:
df = pd.read_csv('C:/Users/Sreerag/Documents/ML_chellange/Detection-of-Breast-Abnormalities/Detection-of-Breast-Abnormalities-from-Mammogram/Main/Data/Parse_data.txt', sep=' ', header=None)

In [53]:
df.head(5)  # Display the first 5 rows of the DataFrame

,0,1,2,3,4,5,6
0,mdb001,G,CIRC,B,535,425,197.0
1,mdb002,G,CIRC,B,522,280,69.0
2,mdb003,D,NORM,NaN,NaN,NaN,NaN
3,mdb004,D,NORM,NaN,NaN,NaN,NaN
4,mdb005,F,CIRC,B,477,133,30.0


| Index | Column Name        |
|-------|--------------------|
| 0     | ImageID            |
| 1     | BreastDensity      |
| 2     | AbnormalityType    |
| 3     | Assessment         |
| 4     | X                  |
| 5     | Y                  |
| 6     | Radius             |


In [54]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 330 entries, 0 to 329
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       330 non-null    object 
 1   1       330 non-null    object 
 2   2       330 non-null    object 
 3   3       123 non-null    object 
 4   4       122 non-null    object 
 5   5       122 non-null    object 
 6   6       119 non-null    float64
dtypes: float64(1), object(6)
memory usage: 18.2+ KB
